In [ ]:
! export TOKENIZERS_PARALLELISM=True
! echo $TOKENIZERS_PARALLELISM

! export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512'

In [1]:
# change tabula to tabula_middle_padding to test middle padding method
from tabula import Tabula 
import pandas as pd

data = pd.read_csv("Real_Datasets/Insurance_compressed.csv")

categorical_columns = ["sex", "children", "sm", "region"]
model = Tabula(
    llm='distilgpt2', 
    experiment_dir = "insurance_training", 
    batch_size=1, 
    epochs=400, 
    categorical_columns = categorical_columns
)

# Comment this block out to test tabula starting from randomly initialized model.
# Comment this block out when uses tabula_middle_padding
import torch
model.model.load_state_dict(torch.load("pretrained-model/model.pt"), strict=False)

model.fit(data)

import torch
torch.save(model.model.state_dict(), "insurance_training/model_400epoch.pt")

2024-03-13 11:06:42.722821: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-13 11:06:42.775475: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-13 11:06:43.647108: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 23.68 GiB of which 9.50 MiB is free. Process 3345811 has 11.58 GiB memory in use. Process 521598 has 2.87 GiB memory in use. Process 521862 has 2.87 GiB memory in use. Process 523781 has 2.87 GiB memory in use. Process 1155050 has 2.87 GiB memory in use. Including non-PyTorch memory, this process has 584.00 MiB memory in use. Of the allocated memory 304.23 MiB is allocated by PyTorch, and 25.77 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
synthetic_data = model.sample(n_samples=1338)
synthetic_data.to_csv("insurance_400epoch.csv", index=False)